# import necessary packages

In [1]:
!pip install pyyaml h5py
!pip install transformers
!pip install scipy
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel
import tensorflow as tf
from transformers import RobertaTokenizer, RobertaModel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# import data & cleaning

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
worksheet = gc.open('PennsylvaniaMidterms11Aug11Oct2').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
import pandas as pd
df=pd.DataFrame.from_records(rows)

new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header
df['SentimentScore']= pd.to_numeric(df['SentimentScore'])
df['Subjectivity']= pd.to_numeric(df['Subjectivity'])
df['Sarcasm']= pd.to_numeric(df['Sarcasm'])
df['Faction']= pd.to_numeric(df['Faction'])
final_df=df.loc[df['SentimentScore'].isin([0,1,2])]

In [4]:
import re
import nltk
def clean_content(content):
  content=re.sub('#','',content)
  content = re.sub(r'@[^(JohnFetterman)][^(DrOz)]\w+', '', content)
  content = re.sub('@', '', content)
  content = re.sub(r'\\u\w+', '', content)
  content = re.sub(r"http\S+", "", content)
  content = re.sub("\n", "", content)
  # content = re.sub(r'[^\w\s\.]', '', content)
  # content=' '.join(s for s in content.split() if not any(c.isdigit() for c in s))
  # content=content.lower()
  return content

final_df['cleaned_tweets']=final_df['Content'].apply(clean_content)
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:                    
    return None
def lemmatize_sentence(content,lemmatizer=lemmatizer):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(content))    
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:             
      res_words.append(word.lower())
    else:
      res_words.append(lemmatizer.lemmatize(word.lower(), tag))
  content=" ".join(res_words)
  content = re.sub(r'[^\w\s]', '', content)
  content=' '.join(s for s in content.split() if not any(c.isdigit() for c in s))
  return content
final_df['lemmatized_tweets']=final_df['cleaned_tweets'].apply(lemmatize_sentence)
sentiment=final_df[final_df['SentimentScore']<2]
sentiment=sentiment[sentiment['Faction']<2]
sentiment=sentiment.dropna(subset=['Faction'])
support=sentiment['SentimentScore'].values
faction=sentiment['Faction'].values
raw_sent=[]
for i in range(len(faction)):
  if faction[i]!=support[i]:
    if support[i]==0:
      raw_sent.append(1)
    else:
      raw_sent.append(0)
  else:
    raw_sent.append(support[i])
sentiment['RawSentiment']=raw_sent
sentiment=sentiment[['cleaned_tweets','lemmatized_tweets','RawSentiment']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: SettingWithC

In [5]:
sentiment

,cleaned_tweets,lemmatized_tweets,RawSentiment
1,VOTE FOR JohnFetterman .Republican DrOz will ...,vote for johnfetterman republican droz will no...,1.0
2,the_vello Why does he always look like a SLOB?...,the_vello why do he always look like a slob he...,1.0
3,John Fetterman is a POS! Just a trust found ki...,john fetterman be a po just a trust find kid w...,1.0
4,"DrOz I would love to see a sit-down, face-to-f...",droz i would love to see a sitdown facetoface ...,1.0
5,Baphomet is non binary and babies were slain f...,baphomet be non binary and baby be slay for mo...,1.0
...,...,...,...
1147,JohnFetterman Democrats destroy everything the...,johnfetterman democrat destroy everything they...,1.0
1148,John Fetterman wipes Black Lives Matter sectio...,john fetterman wipe black life matter section ...,1.0
1158,ProudElephantUS That useless bum is no good an...,proudelephantus that useless bum be no good an...,1.0
1391,DrOz JohnFetterman RacistFetterman JohnFetterman,droz johnfetterman racistfetterman johnfetterman,1.0


# set seed

In [6]:
SEED = 42

import os
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import random 
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import tensorflow as tf
tf.random.set_seed(SEED)

# load base model

In [7]:
# load complex model
tokenizer_roberta = RobertaTokenizerFast.from_pretrained("roberta-base")
def tokenize_roberta(data,max_len=128,tokenizer_roberta=tokenizer_roberta) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer_roberta.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

def roberta_model(bert_model, max_len=128):
    
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    accuracy = tf.keras.metrics.BinaryAccuracy()

    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(128, activation='relu')(output)
    output = tf.keras.layers.Dense(1)(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(opt, loss=loss, metrics=accuracy)
    return model

roberta = TFRobertaModel.from_pretrained('roberta-base')
r_model = roberta_model(roberta)
r_model.load_weights('/content/drive/MyDrive/model/sentiment.h5')

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


# define accuracy function

In [8]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
def get_result(target,pred):
  '''returns accuracy, f1, precision, recall'''
  return accuracy_score(target,pred),f1_score(target,pred,average='weighted'),precision_score(target,pred),recall_score(target,pred)

# output score

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

In [10]:
class Vocabulary:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {}
    self.num_words = 0

  def add_word(self, word):
    if word not in self.word2index:
      # First entry of word into vocabulary
      self.word2index[word] = self.num_words
      self.word2count[word] = 1
      self.index2word[self.num_words] = word
      self.num_words += 1
    else:
      # Word exists; increase word count
      self.word2count[word] += 1

In [11]:
from nltk.tokenize import wordpunct_tokenize
vocab=Vocabulary('tweet')
for i in sentiment['lemmatized_tweets']:
  result=wordpunct_tokenize(i)
  for j in result:
    vocab.add_word(j)
def vectorize(input,vocab=vocab):
  master_list=[]
  for i in input:
    result=wordpunct_tokenize(i)
    tweet=[0]*len(vocab.word2index)
    for j in result:
      for k in vocab.word2index:
        if j==k:
          tweet[vocab.word2index[j]]=1
    master_list.append(tweet)
  return master_list

In [12]:
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.naive_bayes import  MultinomialNB
skf=KFold(n_splits=5, shuffle=True, random_state=SEED)
accuracy=[]
f1=[]
precision=[]
recall=[]
for train_index, test_index in skf.split(sentiment['cleaned_tweets'].values,sentiment['RawSentiment'].values):
  X_train, X_test = sentiment['lemmatized_tweets'].values[train_index], sentiment['lemmatized_tweets'].values[test_index]
  X_train_l,X_test_l=sentiment['cleaned_tweets'].values[train_index], sentiment['cleaned_tweets'].values[test_index]
  y_train, y_test = sentiment['RawSentiment'].values[train_index],sentiment['RawSentiment'].values[test_index]
  #svm
  svm_model=svm.SVC()
  train_vectors = vectorizer.fit_transform(X_train)
  test_vectors = vectorizer.transform(X_test)
  s_train = svm_model.fit(train_vectors,y_train).predict(train_vectors)
  s_pred=svm_model.predict(test_vectors)
  #bayes
  bayes= MultinomialNB()
  train_vectors = vectorize(X_train_l)
  test_vectors = vectorize(X_test_l)
  b_train=bayes.fit(train_vectors,y_train).predict(train_vectors)
  b_pred=bayes.predict(test_vectors)
  #roberta
  train_input_ids, train_attention_masks = tokenize_roberta(X_train, 128)
  result_roberta = r_model.predict([train_input_ids,train_attention_masks])
  result_roberta=tf.math.sigmoid(result_roberta)
  result_roberta=tf.reshape(result_roberta,(result_roberta.shape[0],)).numpy()
  r_train=[]
  for i in result_roberta:
    if i<=0.5:
      r_train.append(0)

    else:
      r_train.append(1)
  test_input_ids, test_attention_masks = tokenize_roberta(X_test, 128)
  result_roberta = r_model.predict([test_input_ids,test_attention_masks])
  result_roberta=tf.math.sigmoid(result_roberta)
  result_roberta=tf.reshape(result_roberta,(result_roberta.shape[0],)).numpy()
  r_pred=[]
  for i in result_roberta:
    if i<=0.5:
      r_pred.append(0)

    else:
      r_pred.append(1)
  one_hot_train=np.transpose(np.stack([s_train,b_train,r_train]))
  one_hot_pred=np.transpose(np.stack([s_pred,b_pred,r_pred]))
  meta_model=svm.SVC()
  y_pred=meta_model.fit(one_hot_train,y_train).predict(one_hot_pred)
  from sklearn import svm
  a,b,c,d=get_result(y_test,y_pred)
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

6/6 [==============================] - 74s 12s/step


In [13]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.8001544309812308, f1: 0.7890080096839339, precision: 0.8081734020001925, recall: 0.9271071496517813


In [14]:
from sklearn.linear_model import LogisticRegression
accuracy=[]
f1=[]
precision=[]
recall=[]
for train_index, test_index in skf.split(sentiment['cleaned_tweets'].values,sentiment['RawSentiment'].values):
  X_train, X_test = sentiment['lemmatized_tweets'].values[train_index], sentiment['lemmatized_tweets'].values[test_index]
  X_train_l,X_test_l=sentiment['cleaned_tweets'].values[train_index], sentiment['cleaned_tweets'].values[test_index]
  y_train, y_test = sentiment['RawSentiment'].values[train_index],sentiment['RawSentiment'].values[test_index]
  #svm
  svm_model=svm.SVC()
  train_vectors = vectorizer.fit_transform(X_train)
  test_vectors = vectorizer.transform(X_test)
  s_train = svm_model.fit(train_vectors,y_train).predict(train_vectors)
  s_pred=svm_model.predict(test_vectors)
  #bayes
  bayes= MultinomialNB()
  train_vectors = vectorize(X_train_l)
  test_vectors = vectorize(X_test_l)
  b_train=bayes.fit(train_vectors,y_train).predict(train_vectors)
  b_pred=bayes.predict(test_vectors)
  #roberta
  train_input_ids, train_attention_masks = tokenize_roberta(X_train, 128)
  result_roberta = r_model.predict([train_input_ids,train_attention_masks])
  result_roberta=tf.math.sigmoid(result_roberta)
  result_roberta=tf.reshape(result_roberta,(result_roberta.shape[0],)).numpy()
  r_train=result_roberta
  test_input_ids, test_attention_masks = tokenize_roberta(X_test, 128)
  result_roberta = r_model.predict([test_input_ids,test_attention_masks])
  result_roberta=tf.math.sigmoid(result_roberta)
  result_roberta=tf.reshape(result_roberta,(result_roberta.shape[0],)).numpy()
  r_pred=result_roberta
  train=np.transpose(np.stack([s_train,b_train,r_train]))
  pred=np.transpose(np.stack([s_pred,b_pred,r_pred]))
  meta_model=LogisticRegression(random_state=SEED)
  y_pred=meta_model.fit(train, y_train).predict(pred)
  a,b,c,d=get_result(y_test,y_pred)
  accuracy.append(a)
  f1.append(b)
  precision.append(c)
  recall.append(d)

6/6 [==============================] - 75s 12s/step


In [15]:
print(f'accuracy: {sum(accuracy)/len(accuracy)}, f1: {sum(f1)/len(f1)}, precision: {sum(precision)/len(precision)}, recall: {sum(recall)/len(recall)}')

accuracy: 0.805624851508672, f1: 0.7897912091049711, precision: 0.8002062515955609, recall: 0.9524818580292822
